# Cogs 108 Final Project

Bernard Suwirjo

Ben Hazarian

Cesar Reyes

Eric Jiang

Gauthier Pred'Astor Gris

In [2]:
# Necessary Imports
import numpy as np
import pandas as pd

In [7]:
# Load Datasets

# Baseball Dataset
NFL_frame = pd.read_csv('data/NFL_16_17.csv')
NFL_frame.head()

,Unnamed: 0,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,Pts,Pts.1,YdsW,TOW,YdsL,TOL,Year
0,0,1,Thu,September 8,8:40PM,Denver Broncos,NaN,Carolina Panthers,boxscore,21.0,20.0,307.0,3.0,333.0,1.0,2016
1,1,1,Sun,September 11,1:04PM,Green Bay Packers,@,Jacksonville Jaguars,boxscore,27.0,23.0,294.0,0.0,348.0,1.0,2016
2,2,1,Sun,September 11,1:04PM,Baltimore Ravens,NaN,Buffalo Bills,boxscore,13.0,7.0,308.0,1.0,160.0,0.0,2016
3,3,1,Sun,September 11,1:05PM,Kansas City Chiefs,NaN,San Diego Chargers,boxscore,33.0,27.0,413.0,1.0,388.0,0.0,2016
4,4,1,Sun,September 11,1:05PM,Tampa Bay Buccaneers,@,Atlanta Falcons,boxscore,31.0,24.0,371.0,1.0,374.0,0.0,2016


Data cleaning

In [14]:
NFL_frame.drop(columns=['Day','Time','Date'])

,Unnamed: 0,Week,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,Pts,Pts.1,YdsW,TOW,YdsL,TOL,Year
0,0,1,Denver Broncos,NaN,Carolina Panthers,boxscore,21.0,20.0,307.0,3.0,333.0,1.0,2016
1,1,1,Green Bay Packers,@,Jacksonville Jaguars,boxscore,27.0,23.0,294.0,0.0,348.0,1.0,2016
2,2,1,Baltimore Ravens,NaN,Buffalo Bills,boxscore,13.0,7.0,308.0,1.0,160.0,0.0,2016
3,3,1,Kansas City Chiefs,NaN,San Diego Chargers,boxscore,33.0,27.0,413.0,1.0,388.0,0.0,2016
4,4,1,Tampa Bay Buccaneers,@,Atlanta Falcons,boxscore,31.0,24.0,371.0,1.0,374.0,0.0,2016
5,5,1,Minnesota Vikings,@,Tennessee Titans,boxscore,25.0,16.0,301.0,0.0,316.0,3.0,2016
6,6,1,Oakland Raiders,@,New Orleans Saints,boxscore,35.0,34.0,486.0,0.0,507.0,1.0,2016
7,7,1,Cincinnati Bengals,@,New York Jets,boxscore,23.0,22.0,381.0,1.0,340.0,1.0,2016
8,8,1,Philadelphia Eagles,NaN,Cleveland Browns,boxscore,29.0,10.0,403.0,0.0,288.0,1.0,2016
9,9,1,Houston Texans,NaN,Chicago Bears,boxscore,23.0,14.0,344.0,1.0,258.0,1.0,2016
